# Compsci 690V Homework 6
This homework is done by **Devdhar Patel**.

We pick the mini challenge 1 of VAST Challenge 2008 for this homework. 

**Description:** The Paraiso movement is controversial and is having considerable social impact in a specific area of the world. We have extracted a segment of the Paraiso (the movement) Wikipedia edits page. Please note this is not the Paraiso Manifesto Wiki page which is part of the background materials, but a related different page. Please use visual analytics to describe the social relationships of the editors (those that have edited/modified the Wikipedia page) as they are reflected in these files.

**Note**: Some parts of this homework are from Homework 5. The homework 6 subsections are clearly marked.



To obtain a dataset similar to the one given in the mini challenge, we scraped the edits of the wikipedia page **"2016–17 Kashmir unrest"** (link: https://en.wikipedia.org/w/index.php?title=2016%E2%80%9317_Kashmir_unrest&limit=500&action=history). We chose this dataset since it is a recent revolutionary movement simirarly to the one described in the challenge. We scraped the text using the script in the file **getEdits.py**.

We parsed the text of each commit into a dataframes with the columns:
* **timestamp** - the timestamp of the edit
* **user** - the user or ip of the editor
* **minorEdit** - True or False - if the edit is minor
* **pageLength** - Length of the page after the edit (in Bytes)
* **editDiff** - bytes changed by the edit
* **comment** - comment or description of the edit
* **tags**
* **entireEdit** - entire raw edit text

In [4]:
import numpy as np
import pandas as pd
import re
from bokeh.plotting import figure
from bokeh.io import output_notebook, show,output_file,install_notebook_hook
from bokeh.models import ColumnDataSource, Select,LabelSet, HoverTool, DatetimeAxis, TapTool, CustomJS, BoxZoomTool, PanTool
from bokeh.models import WheelZoomTool, UndoTool, RedoTool, ResetTool, ZoomInTool, ZoomOutTool
import nltk
import re
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans,DBSCAN
import pprint
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from bokeh.models.widgets import DataTable, TableColumn, Div, RangeSlider, Slider, Select
from bokeh.layouts import Row,widgetbox, Column
from sklearn.manifold import MDS
from sklearn.decomposition import TruncatedSVD
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
from bokeh.palettes import Category10
import math

In [5]:


file = open('data/wiki sample data.txt','r', errors='ignore')

lines = file.readlines()
#print (lines)

df = pd.DataFrame(columns=['timestamp','user','minorEdit','pageLength','editDiff','comment','tags','entireEdit'])
i=0
for line in lines:
        entireEdit = line
        #split via brackets
        tokens = re.split('\(|\)',line)
        timestamp = tokens[2].split('?')[0]
        timestamp = pd.to_datetime(timestamp)
        #split by ?
        user = tokens[2].split('?')[1:]
        user = ''.join(user)
        user = user[1:-1]
        m = tokens[4]
        m= m.split('.')[0]
        m= True if (m=='? m ') else False
        byte = tokens[5].split(' ')[0]
        byte = int(byte.replace(',',''))
        change = tokens[7]
        change = int(change.replace(',',''))
        comment = re.split('\. \. ',line)[-1]
        tagComment = comment.split('(undo)')
        comment = tagComment[0][1:-2]
        #undo is not at the end
        if (len(tagComment)>1):
            tag = tagComment[1][2:-2]
            tag = tag.split(':')
            if (len(tag)>1):
                tag = tag[1][1:]
                tag = tag.split(',')
            if (tag[0]==''):
                tag =[]
        else:
            tag=''

        df.loc[i]=[timestamp,user,m,byte,change,comment,tag,entireEdit]
        i+=1

print (df.shape)

(1338, 8)


Next we look at the users who have made the most edits:

In [6]:
contributions = df['user'].value_counts()
print(contributions.head())

DinoBambinoNFS     437
Kautilya3          101
Support2016         56
117.214.245.178     29
Thnidu              26
Name: user, dtype: int64


Some of the edits have also been done by wikipedia bots. These bots usally have **'Bot'** at the end of their names:

In [7]:
for name in contributions.index.tolist():
    if('Bot' in name):
        print(name, contributions[name])

MusikBot 2
Bender the Bot 2
ClueBot NG 2
KolbertBot 1


We look at the edits that we just reverts of previous commits:

In [8]:
revertEdits = []
for row in df.iterrows():
    if('Revert' in row[1]['comment'] or 'revert' in row[1]['comment']):
        revertEdits.append(row)

print('Number of edits that are reverts: ',len(revertEdits))



Number of edits that are reverts:  83


## Method 1: Term Frequency-Inverse document Frequency feature extraction and clustering
We will use TF-idf feature extraction and cluster the users using Kmeans clustering to see if there are any distinct clusters forming. To do that, we first create a dataframe **userWiseComments** to store all the comments from all the edits of a user. 

In [9]:
def getUserWiseComments(dataframe):
    userWiseComments = pd.DataFrame(columns=['user','allComments'])
    users = list(set(dataframe['user']))
    i = 0
    for user in users:
        userRows = df.loc[df['user'] == user]
        allComments = ''
        for row in userRows.iterrows():
            allComments += row[1]['comment']
        userWiseComments.loc[i] = [user,allComments]
        i += 1
    return userWiseComments
userWiseComments = getUserWiseComments(df)


We will now create a tfidf matrix using the TfidfVectorizer and creating english stems of tokens for all comments of each user:

In [10]:
stemmer = SnowballStemmer("english")

#take comments from dataframe
commentsDf = userWiseComments['allComments']

#token given text and find their stems
def tokenizeAndStem(text):
    CommentsTokens=[]
    #for userComments in commentsDf:
    CommentsTokens = (nltk.word_tokenize(text))
    #filter out punctuations and numeric tokens
    filtered_tokens = []
    for token in CommentsTokens:
        if re.search('[a-zA-Z0-9]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


We can see that there are some similarities in the features in each cluster. We try to plot it using dimensionality reduction via manifold reduction:

## Homework 6 Visualization 2:

In [11]:
def get_colors(clusters):
    colors=[]
    for i in clusters:
        colors.append(Category10[10][i])
        
    return colors



def modify_doc(doc):
    def update(attr, old, new):
        layout.children[2] = create_figure()
    
        
    range_slider = RangeSlider(start=1,end=10, value=(1,4), step=1, title="N-grams Range")
    range_slider.on_change('value', update)
    cluster_slider = Slider(start=1,end=10, value=3, step=1, title="Clusters")
    cluster_slider.on_change('value', update)
    def create_figure():
        tfidf_vectorizer = TfidfVectorizer(max_df=1,
                                 min_df=0, stop_words='english',
                                 use_idf=False,tokenizer=tokenizeAndStem, ngram_range=(int(range_slider.value[0]),int(range_slider.value[1])))  #creating features by taking 3 words at a time

        tfidf_matrix = tfidf_vectorizer.fit_transform(commentsDf)
        num_clusters = cluster_slider.value
        km = KMeans(n_clusters=num_clusters)
        km.fit(tfidf_matrix)

        clusters = km.labels_.tolist()
        colors = get_colors(clusters)
        mds = MDS(n_components=2, random_state=1)
        pos = mds.fit_transform(tfidf_matrix.toarray())  # shape (n_components, n_samples)
        xs, ys = pos[:, 0], pos[:, 1]
        source = ColumnDataSource(data=dict(x=xs,y=ys,colors = colors, user = list(userWiseComments['user']), 
                                   comment = userWiseComments['allComments']))
        hover = HoverTool(tooltips=[
            ("user",'@user'),
            ("comment","@comment")
        ])
        b=figure( plot_height=500, plot_width=900, title='clustering groups based on edits', tools = [hover])
        b.circle('x','y',fill_color='colors',line_color='colors',source=source,size=10)
        return b
    
    p = create_figure()
    layout = Column(range_slider,cluster_slider,p)
    doc.add_root(layout)

output_notebook()    
handler = FunctionHandler(modify_doc)
app = Application(handler)
show(app,notebook_url='localhost:8888')

Loading BokehJS ...

### Result:
As we can see that this method is not very successful in grouping users and there are no clear clusters. This method would work better for actual text data and not comments in edits. However, we can see that on decreasing the range of n-grams, we the graph spread reduces. Therefore, increasing the range of n-grams analyzed might help. Unfortunately, we are working with very limited number of words in each comment and therefore it is not possible.

### Method 2: Creating groups via reverts and mentions

In this method we hypothesize that the users will be split into 4 major group:
1. Bots.
2. Neutral users (unbiased).
3. Users aligned with Government of India, Indian Army and Central Reserve Police.
4. Users aligned with Kashmiri protesters and separatists.

Wikipedia bots have Bot at the end of their name, therefore getting group one is easy:

In [12]:
group1 = []
group2 = []
group3 = []
group4 = []
data = df
for name in contributions.index.tolist():
    if('bot' in name or 'Bot' in name):
        group1.append(name)
        data = data.loc[df['user']!= name]
    
pprint.pprint(group1)

['Yobot',
 'GreenC bot',
 'MusikBot',
 'Bender the Bot',
 'BG19bot',
 'ClueBot NG',
 'KolbertBot']


For group 3 and group 4 we analyze the reverts and mentions of usernames in the comments and try to find the opposing members from that. We extract phrases in which the usernames have been mentioned: 

In [13]:
userWiseComments2 = getUserWiseComments(data)
userMentions = pd.DataFrame(columns=['user','mention','phrase', 'to'])
listOfUserNames = list(userWiseComments2['user'])
for row in userWiseComments2.iterrows():
    tokens = row[1]['allComments'].split()
    for name in listOfUserNames:
        indexes = [i for i,token in enumerate(tokens) if token==name]
        for i in indexes:
            start = max(0, i-5)
            phrase = " ".join(tokens[start:i+1])
            printText = ""
            to = False
            for word in tokens[start:i+1]:
                if(word == 'to' or word == 'To'):
                    printText +="\033[43;3m"+word+"\033[m "
                    to = True
                else:
                    printText +=word + " "
            print(printText)
            userMentions.loc[len(userMentions)] = [row[1]['user'],name,phrase,to]
            
            

Undid revision 730509214 by Kautilya3 
Undid revision 742684927 by Fisaannath22 
beforeReverted to revision 741986450 by Kautilya3 
alternative nameUndid revision 741101224 by Enaam1232 
Reverted 1 pending edit by Jammukashmirwiki 
Kautilya3Reverted 1 pending edit by Ernasir 
Reverted 1 pending edit by 27.124.20.230 
titleReverted 1 pending edit by 27.124.20.230 
for any figuresReverted edits by 103.227.54.88 
(talk): WP:NOTNEWS. (TW)Reverted edits by 39.46.245.15 
Undid revision 801411509 by Mfarazbaig 
Tyler DurdenUndid revision 751079878 by Dscdscdsc11321 
WP:NPOV??May 2017Reverted 1 edit by Amirk94391 
in interimUndid revision 738200822 by Mar4d 
WP:Terrorist. (TW)Undid revision 730644572 by 1.186.211.178 
guns: expandReverted 2 edits by 107.77.169.3 
International angle.Undid revision 733204001 by DinoBambinoNFS 
Reverted to revision 760253021 by Kautilya3 
3dsxl 
thousands.Reverted 2 pending edits by StrivingAbdullah 
Reverted 2 pending edits by 114.143.207.162 
infoReverted 1 pe

From the above phrases, we can see that when the word to is included in the phrase, the mention is done in a positive sense. Otherwise it is done in a negative sense. We will attempt to create a relationship matrix on the following criteria: 
1. if the mention is used in a positive way, we will add 1 to the relationship
2. if the mention is used in a negative way, we will subtract 1 to the relationship
3. All relationships start at 0

In [14]:
relationshipMatrix = {}
for name1 in listOfUserNames:
    relationshipMatrix[name1] = {}
    for name2 in listOfUserNames:
        relationshipMatrix[name1][name2] = 0
        
for row in userMentions.iterrows():
    if(row[1]['to'] == True):
        if(row[1]['user']<row[1]['mention']):
            relationshipMatrix[row[1]['user']][row[1]['mention']] += 1
        else:
            relationshipMatrix[row[1]['mention']][row[1]['user']] += 1
    else:
        if(row[1]['user']<row[1]['mention']):
            relationshipMatrix[row[1]['user']][row[1]['mention']] -= 1
        else:
            relationshipMatrix[row[1]['mention']][row[1]['user']] -= 1

relationships = pd.DataFrame(columns=['user1','user2','relationship'])
i=0
for key1,dictionary in relationshipMatrix.items():
        for key2,value in dictionary.items():
            if(value != 0):
                relationships.loc[i] = [key1,key2,value]
                i+=1

## Homework 6 Visualization 3:

In this visualization, we explore the relationship matrix I built for the second method of grouping. The relationship matrix shows the relation of each user with other users. The more green a pixel is the more positive the relationship is and the more red a pixel is the more negative a relationship is. I have removed the users who do not have any kind of relationship with any other users to make the graph less empty. On clicking each of the pixels, we can view a summary of the relationship between the two users the pixel represents.  

In [15]:
green =['#AFF7A1','#5FF042','#23970C','#092603'] 
red = ['#FDBBB0','#F9674D','#DA2607','#8B1804']
heatmap = pd.DataFrame(columns = ['x','y','value','color','phrases'])


for key1,dictionary in relationshipMatrix.items():
        for key2,value in dictionary.items():
            if(value != 0):
                if(value < 0):
                    color = red[abs(value) - 1]
                else:
                    color = green[abs(value) - 1]
                userRows = userMentions.loc[(userMentions['user'] == key1) & (userMentions['mention'] == key2)]
                mentionRows = userMentions.loc[(userMentions['mention'] == key1) & (userMentions['user'] == key2)]
                phrases = list(userRows['phrase']) + list(mentionRows['phrase'])
                heatmap.loc[len(heatmap)] = [key1,key2,value,color,phrases]
                heatmap.loc[len(heatmap)] = [key2,key1,value,color,phrases]
             
c=figure( plot_height=900, plot_width=900, title='Relationship Heatmap',y_range=list(set(heatmap['y'])),
         x_range=list(set(heatmap['y'])), 
         tools = [TapTool(),BoxZoomTool(), PanTool(),WheelZoomTool(), UndoTool(), RedoTool(), ResetTool(), ZoomInTool(), ZoomOutTool()])
source = ColumnDataSource(data=dict(
    x=heatmap['x'],
    y=heatmap['y'],
    color = heatmap['color'],
    value = heatmap['value'],
    phrases=  heatmap['phrases']
))
c.yaxis.major_label_text_font_size = "6pt"
c.xaxis.major_label_text_font_size = "6pt"
c.xaxis.major_label_orientation = math.pi/2
c.square('x','y',color='color',source=source,size=7)

code2 = """
    console.log('abc')
    var data = source.data,
    selected = source.selected['1d']['indices'],
    select_inds = selected[0];
    console.log('abc')
    heading = "<p>Relationship between <b>"+data['x'][select_inds]+"</b> and <b>" +data['y'][select_inds]+ "</b>:</p>"
    value = "<p><b>Value: </b>"+data['value'][select_inds]+"</p>"
    phraseHeading = "<p><b>Phrases:</b></p>"
    phrases = ""
    for(var i = 0; i< data['phrases'][select_inds].length; i++){
        phrases = phrases + '<p>' +data['phrases'][select_inds][i]+ '</p>'
    }
    div.text = heading+value+phraseHeading+ phrases
    console.log(div.text)
"""

div2 = Div(text="",
width=400, height=100)
source.callback = CustomJS(args={'source':source, 'div':div2},code=code2)
show(Column(c,div2))

**Observation:** I believe that this method is the most accurate of all the three methods. The visualization allows us to explore the phrases on which the relationship is measured and the value of the relationship. This visualization provides a good insight on how we are calculating the relationships and on the intraction of each user with other users.

To create groups, we start by taking the most active users in the **relationships** dataframe and grouping according to them.

In [16]:
relationshipUsers = list(relationships['user1'])+list(relationships['user2'])
uniqueUsers = set(relationshipUsers)
userCount = {}
for user in uniqueUsers:
    userCount[user] = relationshipUsers.count(user)
    
userCount = pd.Series(userCount)
userCount = userCount.sort_values(ascending=False)
for user in userCount.index:
    
    if(user not in group3 and user not in group4):
        group3.append(user)
        
    if(user in group3):
        rows1 = relationships.loc[relationships['user1'] == user]
        rows2 = relationships.loc[relationships['user2'] == user]
        for row in rows1.iterrows():
            user2 = row[1]['user2']
            if(user2 in group3 or user2 in group4):
                continue
            elif(row[1]['relationship']>0):
                group3.append(user2)
            else:
                group4.append(user2)
                
        for row in rows2.iterrows():
            user2 = row[1]['user1']
            if(user2 in group3 or user2 in group4):
                continue
            elif(row[1]['relationship']>0):
                group3.append(user2)
            else:
                group4.append(user2)
        
    if(user in group4):
        rows1 = relationships.loc[relationships['user1'] == user]
        rows2 = relationships.loc[relationships['user2'] == user]
        for row in rows1.iterrows():
            user2 = row[1]['user2']
            if(user2 in group3 or user2 in group4):
                continue
            elif(row[1]['relationship']>0):
                group4.append(user2)
            else:
                group3.append(user2)
                
        for row in rows2.iterrows():
            user2 = row[1]['user1']
            if(user2 in group3 or user2 in group4):
                continue
            elif(row[1]['relationship']>0):
                group4.append(user2)
            else:
                group3.append(user2)

Next we put all the remaining users in **group 2**:

In [17]:
for user in listOfUserNames:
    if(user not in group3 and user not in group4):
        group2.append(user)

The groups that we thus get are:

In [18]:
group1Source = ColumnDataSource(dict(names = group1))
group2Source = ColumnDataSource(dict(names = group2))
group3Source = ColumnDataSource(dict(names = group3))
group4Source = ColumnDataSource(dict(names = group4))


data_table1 = DataTable(source=group1Source, columns=[TableColumn(field="names", title="Group 1 (Bots)")], width=180, height=480,
                        reorderable = False, row_headers= False)
data_table2 = DataTable(source=group2Source, columns=[TableColumn(field="names", title="Group 2 (Neutral)")], width=200, height=480,
                        reorderable = False, row_headers= False)
data_table3 = DataTable(source=group3Source, columns=[TableColumn(field="names", title="Group 3")], width=200, height=480,
                        reorderable = False, row_headers= False)
data_table4 = DataTable(source=group4Source, columns=[TableColumn(field="names", title="Group 4")], width=200, height=480,
                        reorderable = False, row_headers= False)

show(Row(widgetbox(data_table1, width=200),
         widgetbox(data_table2, width=220),
         widgetbox(data_table3, width=220),
         widgetbox(data_table4, width=220)))

### Result:
This method is the most promising of all the methods as it is possible that the two opposite groups would revert and mention each other a lot. Thus this partition is very probable. 

## Method 3: Creating groups via sentiment analysis
In this method we try to analysis the attitude of the users by the sentiment of their comments. We use the **VADER sentiment analysis tool**:

Hutto, C.J. & Gilbert, E.E. (2014). VADER: A Parsimonious Rule-based Model for Sentiment Analysis of Social Media Text. Eighth International Conference on Weblogs and Social Media (ICWSM-14). Ann Arbor, MI, June 2014.

Again we remove all the bots first and the group them by netural, positive and negative. 

In [19]:
green ='#23970C'
red = '#DA2607'
group1 = []
group2 = []
group3 = []
group4 = []
neutrals = pd.DataFrame(columns = ['user','allComments','compound', 'color'])
positives = pd.DataFrame(columns = ['user','allComments','compound', 'color'])
negatives = pd.DataFrame(columns = ['user','allComments','compound', 'color'])
compound = {}
data = df
for name in contributions.index.tolist():
    if('bot' in name or 'Bot' in name):
        group1.append(name)
        data = data.loc[df['user']!= name]
        
sia = SentimentIntensityAnalyzer()
userWiseComments2 = getUserWiseComments(data)
for row in userWiseComments2.iterrows():
    allComments = row[1]['allComments']
    res = sia.polarity_scores(allComments)
    compound[allComments] = res['compound']
    if res['compound'] > 0.2:
        group3.append(row[1]['user'])
        positives.loc[len(neutrals)] = [row[1]['user'],allComments,res['compound'],green]
    elif res['compound'] < -0.2:
        group4.append(row[1]['user'])
        negatives.loc[len(neutrals)] = [row[1]['user'],allComments,res['compound'],red]
    else:
        group2.append(row[1]['user'])
        if(res['compound'] < 0):
            color = red
        else:
            color = green
        neutrals.loc[len(neutrals)] = [row[1]['user'],allComments,res['compound'], color]


## Homework 6 Visualization 4:
In this visualization, we explore the algorithm of sentiment analysis. We can look at 3 different graphs of each of the different groups of users: netutral, positive and negative. Each graph shows the index of the sentiment of each users. We categorize values above 0.2 as positive and below -0.2 as negative and other values as neutral. Upon selecting the group, we are also able to select individual users and look at the sentiment of their individual comments. 

In [20]:
def modify_doc1(doc):
    def update(attr, old, new):
        layout.children[1] = create_figure()
        layout.children[0].children[1] = getUserSelect()
    
    sentimentSelect = Select(title="Select Group:", value="Neutrals", options=["Neutrals", "Positives", "Negatives"])
    sentimentSelect.on_change('value', update)
    def create_figure():
        if(sentimentSelect.value == 'Neutrals'):
            users = list(neutrals['user'])
            source = ColumnDataSource(data=dict(x=list(neutrals['user']),y=list(neutrals['compound']),
                                                colors = list(neutrals['color'])))
        elif(sentimentSelect.value == 'Positives'):
            users = list(positives['user'])
            source = ColumnDataSource(data=dict(x=list(positives['user']),y=list(positives['compound']),
                                                colors = list(positives['color'])))
        else:
            users = list(negatives['user'])
            source = ColumnDataSource(data=dict(x=list(negatives['user']),y=list(negatives['compound']),
                                                colors = list(negatives['color'])))  
            
        
        
        d=figure( plot_height=500, plot_width=900, title='Sentiment Analysis',x_range=users)
        d.vbar(x='x',top='y',bottom=0,fill_color='colors',width=0.5,source=source)
        d.xaxis.major_label_text_font_size = "6pt"
        d.xaxis.major_label_orientation = math.pi/2
        return d
    
    def get_div(value):
        if(sentimentSelect.value == 'Neutrals'):
            data = neutrals
        elif(sentimentSelect.value == 'Positives'):
            data = positives
        else:
            data = negatives
        user = value
        userRow = data.loc[data['user'] == user]
        text = "<h3>"+user+"</h3>"
        text = text + "<p><b>compound:</b> " + str(userRow['compound'].values[0]) + "</p>" 
        text = text + '<table><tr><th>Comment</th><th>Compound</th></tr>'
        comments = df.loc[df['user'] == user]
        for row in comments.iterrows():
            comment = row[1]['comment']
            res = sia.polarity_scores(comment)
            text = text + "<tr><td>"+ comment + "</td><td>"+str(res['compound']) + "</td></tr>"
        text = text + "</table>"
        return text
    
    def setDiv(attr, old, new):
        div.text =  get_div(new)
    
    def getUserSelect():
        if(sentimentSelect.value == 'Neutrals'):
            users = list(neutrals['user'])
        elif(sentimentSelect.value == 'Positives'):
            users = list(positives['user'])
        else:
            users = list(negatives['user'])
        userSelect = Select(title="Select User:", value=users[0], options=users)
        userSelect.on_change('value', setDiv)
        div.text =  get_div(users[0])
        return userSelect
    
    
    p = create_figure()
    div = Div(text="",height=100,width=700)
    userSelect = getUserSelect()
    layout = Column(Row(sentimentSelect,userSelect),p,div)
    doc.add_root(layout)

output_notebook()    
handler1 = FunctionHandler(modify_doc1)
app1 = Application(handler1)
show(app1,notebook_url='localhost:8888')

Loading BokehJS ...

**Observation:** This graph provides a good insight on what text the algoritm classifies as neutral, positive and negative. Upon exploring this visualization more, I observed that some of the text has not been correctly classified. For example the text: "please do not BLINDLY REVERT everything. you r opposition was to a paragraph (that has not even be removed!)" has been classified as positive with a compound value of 0.3802. However, the text is clearly negative. This suggests that we would need to tweak the algorithm a bit to run it on comments of wiki-edits. 

In [21]:
group1Source = ColumnDataSource(dict(names = group1))
group2Source = ColumnDataSource(dict(names = group2))
group3Source = ColumnDataSource(dict(names = group3))
group4Source = ColumnDataSource(dict(names = group4))


data_table1 = DataTable(source=group1Source, columns=[TableColumn(field="names", title="Group 1 (Bots)")], width=180, height=480,
                        reorderable = False, row_headers= False)
data_table2 = DataTable(source=group2Source, columns=[TableColumn(field="names", title="Group 2 (Neutral)")], width=200, height=480,
                        reorderable = False, row_headers= False)
data_table3 = DataTable(source=group3Source, columns=[TableColumn(field="names", title="Group 3 (Positive)")], width=200, height=480,
                        reorderable = False, row_headers= False)
data_table4 = DataTable(source=group4Source, columns=[TableColumn(field="names", title="Group 4 (Negative)")], width=200, height=480,
                        reorderable = False, row_headers= False)

show(Row(widgetbox(data_table1, width=200),
         widgetbox(data_table2, width=220),
         widgetbox(data_table3, width=220),
         widgetbox(data_table4, width=220)))

### Result:
This method is successful in grouping users by their attitudes. However, this method alone would not be able to give the grouping of each faction. Combining this method with method 2 would help improve the result. Overall, method 2 is the most promising method for solving the Vast 2008 Mini challenge 1 